# Import Required Modules

In [ ]:
from NRMS import *
from GATNRMS import *
from config import *
from evaluate import *
from base_dataset import *

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Train Function

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from base_dataset import BaseDataset
from config import NRMSConfig
from config import GATNRMSConfig
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
from config import model_name
from tqdm.notebook import tqdm
import os
from pathlib import Path
from evaluate import evaluate
import importlib
import datetime
from NRMS import *
from GATNRMS import *
import wandb

try:
  if model_name == 'NRMS':
      config = NRMSConfig()
      Model=NRMS
  elif model_name == 'GATNRMS':
      config = GATNRMSConfig()
      Model=GATNRMS
except AttributeError:
    print(f"{model_name} not included!")
    exit()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class EarlyStopping:
    def __init__(self, patience=5):
        self.patience = patience
        self.counter = 0
        self.best_loss = np.Inf

    def __call__(self, val_loss):
        """
        if you use other metrics where a higher value is better, e.g. accuracy,
        call this with its corresponding negative value
        """
        if val_loss < self.best_loss:
            early_stop = False
            get_better = True
            self.counter = 0
            self.best_loss = val_loss
        else:
            get_better = False
            self.counter += 1
            if self.counter >= self.patience:
                early_stop = True
            else:
                early_stop = False

        return early_stop, get_better


def latest_checkpoint(directory):
    if not os.path.exists(directory):
        return None
    all_checkpoints = {
        int(x.split('.')[-2].split('-')[-1]): x
        for x in os.listdir(directory)
    }
    if not all_checkpoints:
        return None
    return os.path.join(directory,
                        all_checkpoints[max(all_checkpoints.keys())])


def train():
    """
    Train model.
    """
    # wandb.init(project="NewsRecommender", name=f"{model_name}-run-1", config=config)
    wandb.init(project="NewsRecommender", name=f"{model_name1}-run", config=config)

    writer = SummaryWriter(
        log_dir=
        f"./runs/{model_name}/{datetime.datetime.now().replace(microsecond=0).isoformat()}{'-' + os.environ['REMARK'] if 'REMARK' in os.environ else ''}"
    )

    if not os.path.exists('checkpoint'):
        os.makedirs('checkpoint')

    try:
        pretrained_word_embedding = torch.from_numpy(
            np.load('/content/data/train/pretrained_word_embedding.npy')).float()
    except FileNotFoundError:
        pretrained_word_embedding = None

    model = Model(config, pretrained_word_embedding).to(device)
    wandb.watch(model)

    print(model)

    dataset = BaseDataset('/content/data/train/behaviors_parsed.tsv',
                          '/content/data/train/news_parsed.tsv')

    print(f"Load training dataset with size {len(dataset)}.")

    dataloader = iter(
        DataLoader(dataset,
                   batch_size=config.batch_size,
                   shuffle=True,
                   num_workers=config.num_workers,
                   drop_last=True,
                   pin_memory=True))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                     lr=config.learning_rate)
    start_time = time.time()
    loss_full = []
    exhaustion_count = 0
    step = 0
    early_stopping = EarlyStopping()

    checkpoint_dir = os.path.join('/content/checkpoint', model_name)
    Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)

    checkpoint_path = latest_checkpoint(checkpoint_dir)
    if checkpoint_path is not None:
        print(f"Load saved parameters in {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path)
        early_stopping(checkpoint['early_stop_value'])
        step = checkpoint['step']
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        model.train()

    for i in tqdm(range(
            1,
            config.num_epochs * len(dataset) // config.batch_size + 1),
                  desc="Training"):
        try:
            minibatch = next(dataloader)
        except StopIteration:
            exhaustion_count += 1
            tqdm.write(
                f"Training data exhausted for {exhaustion_count} times after {i} batches, reuse the dataset."
            )
            dataloader = iter(
                DataLoader(dataset,
                           batch_size=config.batch_size,
                           shuffle=True,
                           num_workers=config.num_workers,
                           drop_last=True,
                           pin_memory=True))
            minibatch = next(dataloader)

        step += 1
        y_pred = model(minibatch["candidate_news"],
                           minibatch["clicked_news"])

        y = torch.zeros(len(y_pred)).long().to(device)
        loss = criterion(y_pred, y)

        loss_full.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            wandb.log({"Train/Loss": loss.item()}, step=step)
            writer.add_scalar('Train/Loss', loss.item(), step)

        if i % config.num_batches_show_loss == 0:
            tqdm.write(
                f"Time {time_since(start_time)}, batches {i}, current loss {loss.item():.4f}, average loss: {np.mean(loss_full):.4f}, latest average loss: {np.mean(loss_full[-256:]):.4f}"
            )

        if i % config.num_batches_validate == 0:
            model.eval()
            val_auc, val_mrr, val_ndcg5, val_ndcg10 = evaluate(
                model, './data/val',
                config.num_workers, 200000)
            wandb.log({
                "Validation/AUC": val_auc,
                "Validation/MRR": val_mrr,
                "Validation/nDCG@5": val_ndcg5,
                "Validation/nDCG@10": val_ndcg10
            }, step=step)
            model.train()


            writer.add_scalar('Validation/AUC', val_auc, step)
            writer.add_scalar('Validation/MRR', val_mrr, step)
            writer.add_scalar('Validation/nDCG@5', val_ndcg5, step)
            writer.add_scalar('Validation/nDCG@10', val_ndcg10, step)
            tqdm.write(
                f"Time {time_since(start_time)}, batches {i}, validation AUC: {val_auc:.4f}, validation MRR: {val_mrr:.4f}, validation nDCG@5: {val_ndcg5:.4f}, validation nDCG@10: {val_ndcg10:.4f}, "
            )

            early_stop, get_better = early_stopping(-val_auc)
            if early_stop:
                tqdm.write('Early stop.')
                break
            elif get_better:
                try:
                    torch.save(
                        {
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict':
                            optimizer.state_dict(),
                            'step':
                            step,
                            'early_stop_value':
                            -val_auc
                        }, f"./checkpoint/{model_name}/ckpt-{step}.pth")
                    wandb.save(f"./checkpoint/{model_name}/ckpt-{step}.pth")
                except OSError as error:
                    print(f"OS error: {error}")


    wandb.finish()
    writer.close()


def time_since(since):
    """
    Format elapsed time string.
    """
    now = time.time()
    elapsed_time = now - since
    return time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

# GAT-NRMS Model

In [ ]:
print('Using device:', device)
print(f'Training model {model_name}')
train()

Using device: cuda:0
Training model GATNRMS


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: avinash-saxena (avinash-saxena-san-jose-state-university). Use `wandb login --relogin` to force relogin


GATNRMS(
  (news_encoder): NewsEncoder(
    (word_embedding): Embedding(70973, 300, padding_idx=0)
    (gat): GraphAttentionLayer(
      (leakyrelu): LeakyReLU(negative_slope=0.2)
    )
    (attention): AdditiveAttention(
      (linear): Linear(in_features=300, out_features=200, bias=True)
    )
  )
  (user_encoder): UserEncoder(
    (gat): GraphAttentionLayer(
      (leakyrelu): LeakyReLU(negative_slope=0.2)
    )
    (attention): AdditiveAttention(
      (linear): Linear(in_features=300, out_features=200, bias=True)
    )
  )
  (click_predictor): DotProductClickPredictor()
)
Load training dataset with size 225201.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Training:   0%|          | 0/3518 [00:00<?, ?it/s]

...

Calculating vectors for users:   0%|          | 0/25 [00:00<?, ?it/s]

/content/evaluate.py:146: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.behaviors.clicked_news.fillna(' ', inplace=True)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potentia

Calculating probabilities:   0%|          | 0/73152 [00:00<?, ?it/s]

Time 01:09:55, batches 3300, validation AUC: 0.6452, validation MRR: 0.2899, validation nDCG@5: 0.3191, validation nDCG@10: 0.3870, 
Time 01:10:25, batches 3400, current loss 0.8661, average loss: 0.8950, latest average loss: 0.8565
Time 01:10:55, batches 3500, current loss 0.8918, average loss: 0.8938, latest average loss: 0.8563


Train/Loss,█▇█▇▆▆▅▄▄▄▅▂▂▄▃▃▄▅▅▁▄▇▃▁▄▂▅▂▆▂▂▁▃▂▂▃▃▄▃▃
Validation/AUC,▁▄▅▆▆▆▆▇▇██
Validation/MRR,▁▅▅▆▇▇▆▇██▇
Validation/nDCG@10,▁▅▅▆▆▇▆▇███
Validation/nDCG@5,▁▅▅▆▆▇▆▇▇██
Train/Loss,0.84757
Validation/AUC,0.64517
Validation/MRR,0.28991
Validation/nDCG@10,0.38695
Validation/nDCG@5,0.31911


In [ ]:
def load_model(Model, model_name):
    checkpoint = torch.load(f"/content/checkpoint/{model_name}/ckpt-3000.pth")

    try:
            pretrained_word_embedding = torch.from_numpy(
            np.load('/content/data/train/pretrained_word_embedding.npy')).float()
    except FileNotFoundError:
            pretrained_word_embedding = None

    model = Model(config, pretrained_word_embedding)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer = torch.optim.Adam(model.parameters(),lr=config.learning_rate)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    step = checkpoint['step']
    early_stop_value = checkpoint['early_stop_value']

    return model, optimizer, step, early_stop_value

In [ ]:
model, optimizer, step, early_step_value = load_model(GATNRMS, "GATNRMS")

<ipython-input-11-03ee41a46515>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/content/checkpoint/GATNRMS/ckpt-3000.pth")


In [ ]:
model

GATNRMS(
  (news_encoder): NewsEncoder(
    (word_embedding): Embedding(70973, 300, padding_idx=0)
    (gat): GraphAttentionLayer(
      (leakyrelu): LeakyReLU(negative_slope=0.2)
    )
    (attention): AdditiveAttention(
      (linear): Linear(in_features=300, out_features=200, bias=True)
    )
  )
  (user_encoder): UserEncoder(
    (gat): GraphAttentionLayer(
      (leakyrelu): LeakyReLU(negative_slope=0.2)
    )
    (attention): AdditiveAttention(
      (linear): Linear(in_features=300, out_features=200, bias=True)
    )
  )
  (click_predictor): DotProductClickPredictor()
)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
def evaluate_model(model, data_path, num_workers=4):
  model.to(device)
  model.eval()
  val_auc, val_mrr, val_ndcg5, val_ndcg10 = evaluate(model, data_path,config.num_workers, 200000)
  print(f"validation AUC: {val_auc:.4f}, validation MRR: {val_mrr:.4f}, validation nDCG@5: {val_ndcg5:.4f}, validation nDCG@10: {val_ndcg10:.4f}")

In [ ]:
evaluate_model(model, '/content/data/test')

Calculating vectors for news:   0%|          | 0/21 [00:00<?, ?it/s]

Calculating vectors for users:   0%|          | 0/25 [00:00<?, ?it/s]

Calculating probabilities:   0%|          | 0/73152 [00:00<?, ?it/s]

validation AUC: 0.6460, validation MRR: 0.2934, validation nDCG@5: 0.3213, validation nDCG@10: 0.3891


# NRMS Model

## Before running below cell change model_name in config.py

In [ ]:
print('Using device:', device)
print(f'Training model {model_name}')
train()

Using device: cuda:0
Training model NRMS


Train/Loss,▇▇▆▄▆▆▆█▅▄▆▄▅▅▆▄▂▃▅▄▅▅▄▃▄▄▅▅▂▂▅▃▁▃▅▁
Train/Loss,0.98898


NRMS(
  (news_encoder): NewsEncoder(
    (word_embedding): Embedding(70973, 300, padding_idx=0)
    (multihead_self_attention): MultiHeadSelfAttention(
      (W_Q): Linear(in_features=300, out_features=300, bias=True)
      (W_K): Linear(in_features=300, out_features=300, bias=True)
      (W_V): Linear(in_features=300, out_features=300, bias=True)
    )
    (additive_attention): AdditiveAttention(
      (linear): Linear(in_features=300, out_features=200, bias=True)
    )
  )
  (user_encoder): UserEncoder(
    (multihead_self_attention): MultiHeadSelfAttention(
      (W_Q): Linear(in_features=300, out_features=300, bias=True)
      (W_K): Linear(in_features=300, out_features=300, bias=True)
      (W_V): Linear(in_features=300, out_features=300, bias=True)
    )
    (additive_attention): AdditiveAttention(
      (linear): Linear(in_features=300, out_features=200, bias=True)
    )
  )
  (click_predictor): DotProductClickPredictor()
)
Load training dataset with size 225201.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Training:   3%|▎         | 100/3518 [00:33<18:53,  3.02it/s]

Time 00:00:33, batches 100, current loss 1.0575, average loss: 1.0764, latest average loss: 1.0764


...

Time 01:20:37, batches 3400, current loss 0.9809, average loss: 0.9507, latest average loss: 0.9091


Training:  99%|█████████▉| 3500/3518 [1:21:10<00:06,  2.92it/s]

Time 01:21:09, batches 3500, current loss 0.9357, average loss: 0.9494, latest average loss: 0.9028


Training: 100%|██████████| 3518/3518 [1:21:15<00:00,  1.39s/it]


Train/Loss,█▇▇▅▇▅▅▅▅▅▅▄▅▃▄▂▃▁▅▄▄▂▄▃▅▂▃▅▆▃▂▃▃▂▃▆▂▃▂▅
Validation/AUC,▁▂▅▅▇▇▇████
Validation/MRR,▁▂▃▅▆▆▇████
Validation/nDCG@10,▁▂▄▅▆▇▇████
Validation/nDCG@5,▁▂▃▅▆▆▇████
Train/Loss,0.94833
Validation/AUC,0.63275
Validation/MRR,0.28576
Validation/nDCG@10,0.37818
Validation/nDCG@5,0.30971
